In [ ]:
# Initate Spark and configure 

In [ ]:
!pip install -U pyspark
from pyspark.sql import SparkSession

In [ ]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

print("Using Apache Spark Version", spark.version)

In [ ]:
## Search Engine Template -> uses spark df to query based on user input
### need to different html -> one is the home page (user input) then the second one is the output screen
#### there is a way to only use one html for simplicity 

In [ ]:
from flask import Flask, request, jsonify, redirect, url_for, render_template
import numpy as np

app = Flask("JSON_OUTPUT")

@app.route('/')
def form():
    return render_template('moreedits.html')
        
@app.route('/submit', methods=['GET','POST'])
def submit():
    if request.method == 'POST':
        types_user = request.form['q']
        price_level_user = request.form['price_level']
        neighborhood_user = request.form['neighborhoods']
        rating_user = request.form['rating']
        
        def cossim(v1, v2):
            dot_product = np.sum(v1 * v2)
            mag_v1 = np.sqrt(np.sum(np.power(v1, 2)))
            mag_v2 = np.sqrt(np.sum(np.power(v2, 2)))
            return dot_product / (mag_v1 * mag_v2 + 0.1)

        query_txt = types_user
        query_df = sc.parallelize([(1,query_txt)]).toDF(['index','Types'])
        query_tok = regexTokFilter.transform(query_df)
        query_vec = model_type.transform(query_tok)
        query_vec = query_vec.select('wordvectors_type').collect()[0][0]
        
        sim_rdd = sc.parallelize((i[0],i[1], i[2],i[3],i[4],i[5],i[6],float(cossim(query_vec, i[7])), i[8]) for i in sparkDF_wv_final)
        sim_df = spark.createDataFrame(sim_rdd).\
            withColumnRenamed('_1', 'Name').\
            withColumnRenamed('_2', 'Address').\
            withColumnRenamed('_3', 'Latitude').\
            withColumnRenamed('_4', 'Longitude').\
            withColumnRenamed('_5', 'Types').\
            withColumnRenamed('_6', 'Price_level').\
            withColumnRenamed('_7', 'Rating').\
            withColumnRenamed('_8', 'Similarity').\
            withColumnRenamed('_9', 'Neighborhood').\
            orderBy("Similarity", ascending=False)

    
    pandas_df = sim_df.toPandas()
    df_filtered = pandas_df[pandas_df['Price_level'] == int(price_level_user)]
    df_filtered1 = df_filtered[df_filtered['Rating'] == int(rating_user)]
    df_filtered2 = df_filtered1[df_filtered1['Neighborhood'] == neighborhood_user]
    html_table = df_filtered2.head(10).to_html(classes='table')
    return render_template('onlytable.html', table=html_table)

@app.route('/output')
def output():
    # render the output HTML page
    return render_template('onlytable.html',table=processed_data)

app.run(host='localhost', port=7030)